In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
from ase import units
from scipy.integrate import trapezoid

In [2]:
def integrate_switching(
    df_log: pd.DataFrame,
    equil_time: int = 20000,
    switch_time: int = 30000,
    return_E_diss: bool = False,
):
    fwd_start, fwd_end = equil_time, equil_time + switch_time
    rev_start, rev_end = 2 * equil_time + switch_time, 2 * equil_time + 2 * switch_time
    grad, lamda = df_log["lambda_grad"], df_log["lambda"]
    W_fwd = trapezoid(grad[fwd_start:fwd_end], lamda[fwd_start:fwd_end])
    W_rev = trapezoid(grad[rev_start:rev_end], lamda[rev_start:rev_end])
    if return_E_diss:
        return (W_fwd - W_rev) / 2, (W_fwd + W_rev) / 2
    return (W_fwd - W_rev) / 2  # free energy difference


def analyze_frenkel_ladd(
    base_path: Path,
    temp: float,
    equil_time: int = 20000,
    switch_time: int = 30000,
    verbose: bool = False,
):
    T = temp
    k = np.load(base_path / "spring_constants.npy")

    mass = np.load(base_path / "masses.npy")
    omega = np.sqrt(k / mass)
    n_atoms = len(mass)

    # 1. Perfect crystal
    df_log = pd.read_csv(base_path / "observables.csv")
    volume = df_log["volume"].values[0]
    if verbose:
        _, E_diss_perfect = integrate_switching(
            df_log, equil_time, switch_time, return_E_diss=True
        )
    delta_F = integrate_switching(df_log, equil_time, switch_time)
    F_E = 3 * units.kB * T * np.mean(np.log(units._hbar * omega / (units.kB * T)))
    PV = volume * 1.01325 * units.bar
    G_perfect = delta_F + F_E + PV

    # 2. Defective crystal
    df_log = pd.read_csv(base_path / "observables_defect.csv")
    volume = df_log["volume"].values[0]
    if verbose:
        _, E_diss_defect = integrate_switching(
            df_log, equil_time, switch_time, return_E_diss=True
        )
    delta_F = integrate_switching(df_log, equil_time, switch_time)
    F_E = 3 * units.kB * T * np.mean(np.log(units._hbar * omega / (units.kB * T)))
    PV = volume * 1.01325 * units.bar
    G_defect = delta_F + F_E + PV
    G_v = G_defect * (n_atoms - 1) - G_perfect * (n_atoms - 1)

    # 3. Partial FL
    df_log = pd.read_csv(base_path / "observables_FL.csv")
    if verbose:
        _, E_diss_FL = integrate_switching(
            df_log, equil_time, switch_time, return_E_diss=True
        )
    delta_G = integrate_switching(df_log, equil_time, switch_time)
    # delta_G * N = (G_defect * N-1 + F_E) - G_perfect * N
    G_defect_alt = (delta_G * n_atoms - F_E + G_perfect * n_atoms) / (n_atoms - 1)
    G_v_alt = G_defect_alt * (n_atoms - 1) - G_perfect * (n_atoms - 1)

    if verbose:
        return G_perfect, G_defect, delta_G, E_diss_perfect, E_diss_defect, E_diss_FL
    return G_v, G_v_alt

In [3]:
result_path = Path("../data/results/vacancy")
temp_range = [50, 100, 150, 200]

G_v_all, G_v_std_all = [], []
G_v_alt_all, G_v_alt_std_all = [], []
for temp in temp_range:
    G_v_list = []
    G_v_alt_list = []
    E_diss_perfect_list = []
    E_diss_defect_list = []
    E_diss_FL_list = []
    for i in range(4):
        base_path = result_path / f"Fe_5x5x5_{temp}K/{i}"
        G_v, G_v_alt = analyze_frenkel_ladd(base_path, temp=temp, verbose=False)
        G_v_list.append(G_v)
        G_v_alt_list.append(G_v_alt)
    G_v = np.mean(G_v_list)
    G_v_std = np.std(G_v_list)
    G_v_alt = np.mean(G_v_alt_list)
    G_v_alt_std = np.std(G_v_alt_list)
    print(f"G_v ({temp} K) = {G_v:.4f} ± {G_v_std:.4f} eV")
    print(f"G_v FL ({temp} K) = {G_v_alt:.4f} ± {G_v_alt_std:.4f} eV")
    G_v_all.append(G_v)
    G_v_std_all.append(G_v_std)
    G_v_alt_all.append(G_v_alt)
    G_v_alt_std_all.append(G_v_alt_std)

G_v (50 K) = 1.5513 ± 0.0063 eV
G_v FL (50 K) = 1.5582 ± 0.0014 eV
G_v (100 K) = 1.5475 ± 0.0060 eV
G_v FL (100 K) = 1.5405 ± 0.0018 eV
G_v (150 K) = 1.5232 ± 0.0276 eV
G_v FL (150 K) = 1.5188 ± 0.0029 eV
G_v (200 K) = 1.5110 ± 0.0283 eV
G_v FL (200 K) = 1.5003 ± 0.0072 eV
